In [1]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain_experimental.text_splitter import SemanticChunker
from langchain.vectorstores import Chroma
from langchain.document_loaders import Docx2txtLoader
import os


In [2]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
import openai
openai.api_key = os.environ['OPENAI_API_KEY']


In [ ]:
# Load file
document_path = "../data/Evaluation Sets/Raptor Contract.docx"
loader = Docx2txtLoader(document_path)
documents = loader.load()
documents


In [4]:
text_splitter = SemanticChunker(OpenAIEmbeddings())
texts = text_splitter.split_documents(documents)

/home/elias/miniconda/envs/langc/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
texts

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

# Define the desired character chunk size
chunk_size = 300  # Adjust as needed

# Create a CharacterTextSplitter
character_splitter = CharacterTextSplitter(chunk_size=chunk_size)

# Split each semantic chunk into character chunks
character_chunks = []
for semantic_chunk in texts:
    character_chunks.extend(character_splitter.split_documents([semantic_chunk]))

# Now the character_chunks variable contains the list of character-based chunks
print(character_chunks)


In [7]:
embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(character_chunks, embeddings)

In [8]:
# Retrieval Chains:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.as_retriever())


/home/elias/miniconda/envs/langc/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [9]:
query = "How much is the escrow amount??"
qa.run(query)

/home/elias/miniconda/envs/langc/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' The escrow amount is not clearly stated in the given context. It is mentioned multiple times, but the exact amount is not specified.'

In [10]:
query = "What is the purpose of the escrow?"
qa.run(query)


' The purpose of the escrow is to hold funds or assets on behalf of the Sellers in connection with the Contemplated Transactions.'

In [11]:
query = "Are there any conditions to the closing?"
qa.run(query)

' Yes, there are conditions set forth in the agreement that must be met before the closing can occur. These conditions include the delivery of certain documents and the payment of the purchase price.'